In [1]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

In [2]:
import os
import sys

# Add parent directory to path (which is the 'src' directory containing data_loaders.py)
parent_dir = os.path.dirname(os.getcwd())  # This gets the src directory
sys.path.append(parent_dir)

# Now you can import directly
from data_loaders import data_loader

In [3]:
learning_rates = [0.001, 0.0001, 0.01]
metrics = {lr: {'loss': [], 'accuracy': []} for lr in learning_rates}

In [4]:
train_loader, test_loader = data_loader('CIFAR10', 128, 8)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
num_epochs = 10

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Reinitialize the model for each learning rate
    model = torchvision.models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.to(device)

    # Define optimizer with the current learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total

        metrics[lr]['loss'].append(epoch_loss)
        metrics[lr]['accuracy'].append(epoch_accuracy)

        print(f"LR {lr} | Epoch [{epoch+1}/{num_epochs}] | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.2f}%")

    # Save model weights
    model_save_path = f"../trained_model_weights/resnet18_lr_{lr}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model with LR {lr} saved to {model_save_path}")

Training with learning rate: 0.001
LR 0.001 | Epoch [1/10] | Loss: 1.5428 | Accuracy: 43.63%
LR 0.001 | Epoch [2/10] | Loss: 1.1770 | Accuracy: 57.80%
LR 0.001 | Epoch [3/10] | Loss: 1.0221 | Accuracy: 63.81%
LR 0.001 | Epoch [4/10] | Loss: 0.9183 | Accuracy: 67.78%
LR 0.001 | Epoch [5/10] | Loss: 0.8381 | Accuracy: 70.60%
LR 0.001 | Epoch [6/10] | Loss: 0.7833 | Accuracy: 72.44%
LR 0.001 | Epoch [7/10] | Loss: 0.7383 | Accuracy: 74.35%
LR 0.001 | Epoch [8/10] | Loss: 0.6894 | Accuracy: 76.01%
LR 0.001 | Epoch [9/10] | Loss: 0.6684 | Accuracy: 76.75%
LR 0.001 | Epoch [10/10] | Loss: 0.6302 | Accuracy: 77.94%
Model with LR 0.001 saved to ../trained_model_weights/resnet18_lr_0.001.pth
Training with learning rate: 0.0001
LR 0.0001 | Epoch [1/10] | Loss: 1.7484 | Accuracy: 35.86%
LR 0.0001 | Epoch [2/10] | Loss: 1.4565 | Accuracy: 46.66%
LR 0.0001 | Epoch [3/10] | Loss: 1.3205 | Accuracy: 51.87%
LR 0.0001 | Epoch [4/10] | Loss: 1.2284 | Accuracy: 55.47%
LR 0.0001 | Epoch [5/10] | Loss: 1.1

In [18]:
print("Directory created:", os.path.exists("../trained_model_weights"))

Directory created: True


In [19]:
import json
with open("../trained_model_weights/resnet18_metrics.json", "w") as f:
    json.dump(metrics, f)